In [33]:
import numpy as np

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import lightning.pytorch as pl
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import wandb

import sys, os


In [34]:
class MLPModel(pl.LightningModule):

    def __init__(self, d_input: int, d_output: int, d_layers: list,
                 dropout: float = 0.0, lr: float = 1e-2, gamma: float = 1.0):
        super().__init__()
        self.save_hyperparameters()
        
        # Misc parameters
        self.model_type = 'MLP'
        self.lr = lr
        self.gamma = gamma
        self.train_losses = []

        # MLP layers
        self.input_layer = nn.Linear(d_input, d_layers[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(d_layers[i], d_layers[i+1]) for i in range(len(d_layers)-1)])
        self.output_layer = nn.Linear(d_layers[-1], d_output)

        self.activ = nn.ReLU()
        self.loss = nn.MSELoss()


    def forward(self, src: Tensor) -> Tensor:

        x = self.activ(self.input_layer(src))
        for layer in self.hidden_layers:
            x = self.activ(layer(x))
        x = self.output_layer(x)

        return x


    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)

        # Compute loss and save it
        loss = self.loss(outputs, targets)
        self.log("train/loss", loss, sync_dist=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)

        # Compute loss and save it
        loss = self.loss(outputs, targets)
        self.log("valid/loss", loss, sync_dist=True)
        
        return outputs, loss


    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)

        return outputs

    def configure_optimizers(self):
        # optimizer = torch.optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=self.gamma)
        return [optimizer], [scheduler]

In [43]:
class Weather_dataset(torch.utils.data.Dataset):
    def __init__(self, dataset: int = 0):

        # Load dataset from library
        data = np.load(os.path.join('/Users/alberic/Desktop/divers/projects/hvac_opt/weather_model/data/combined_Room1', 
                                    f'normalized_data_{dataset}.npy'))
        
        self.input = data[:len(data)-1]
        self.target = data[1:, 6:8]

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        return ( torch.tensor(self.input[idx]).type(torch.float), torch.tensor(self.target[idx]).type(torch.float) )        
    

class Weather_dataModule(pl.LightningDataModule):

    def __init__(self, batch_train_size: int = 32, batch_valid_size: int = 32):
        super().__init__()
        self.save_hyperparameters()

        self.batch_train_size = batch_train_size
        self.batch_valid_size = batch_valid_size

        self.train_dataset = Weather_dataset(dataset=2)
        self.valid_dataset = Weather_dataset(dataset=3)
        self.test_dataset = Weather_dataset(dataset=4)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_train_size, shuffle=True, num_workers=1)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_dataset, batch_size=self.batch_valid_size, num_workers=1)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_valid_size, num_workers=1)

In [47]:
dataModule.train_dataset.target.shape

(1439, 2)

In [44]:
dataModule = Weather_dataModule(batch_train_size=32, batch_valid_size=32)

model = MLPModel(d_input=29, d_output=2, d_layers=[64, 32, 16, 8, 2], lr=1e-3, gamma=1.0)

trainer = pl.Trainer(
                    accelerator='cpu', devices=1,
                    max_epochs=10, 
                    log_every_n_steps=1,
                    accumulate_grad_batches=1,
                    # logger=wandb_logger,
                    # callbacks=[
                    #             # EarlyStopping(monitor="valid/loss", mode='min', patience=30),
                    #             PredictionLogger(),
                    #             ModelChecker(log_every_nstep=100000)]
                                )

trainer.fit(model=model, datamodule=dataModule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/alberic/.pyenv/versions/3.10.9/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning:

MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.


  | Name          | Type       | Params
---------------------------------------------
0 | input_layer   | Linear     | 1.9 K 
1 | hidden_layers | ModuleList | 2.8 K 
2 | output_layer  | Linear     | 6     
3 | activ         | ReLU       | 0     
4 | loss          | MSELoss    | 0     
---------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/alberic/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/alberic/.pyenv/versions/3.10.9/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'Weather_dataset' on <module '__main__' (built-in)>
